# Inference notenook for [CLIP prefix captioning](https://github.com/rmokady/CLIP_prefix_caption/)

Disclaimer: the authors do not own any rights for the code or data.

In [1]:
#@title Install
# !pip install transformers
# ! pip install git+https://github.com/openai/CLIP.git


In [2]:
# !pip install PyDrive

In [3]:
# !pip install google

In [4]:
# !pip install --user -U nltk

In [5]:
# !pip install google-colab

In [191]:
#@title Imports

import glob
import clip
import os
from torch import nn
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as nnf
import sys
from typing import Tuple, List, Union, Optional
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
# from google.colab import files
import skimage.io as io
import PIL.Image
from IPython.display import Image
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity

tqdm.pandas()


N = type(None)
V = np.array
ARRAY = np.ndarray
ARRAYS = Union[Tuple[ARRAY, ...], List[ARRAY]]
VS = Union[Tuple[V, ...], List[V]]
VN = Union[V, N]
VNS = Union[VS, N]
T = torch.Tensor
TS = Union[Tuple[T, ...], List[T]]
TN = Optional[T]
TNS = Union[Tuple[TN, ...], List[TN]]
TSN = Optional[TS]
TA = Union[T, ARRAY]


D = torch.device
CPU = torch.device('cpu')


def get_device(device_id: int) -> D:
    if not torch.cuda.is_available():
        return CPU
    device_id = min(torch.cuda.device_count() - 1, device_id)
    return torch.device(f'cuda:{device_id}')


CUDA = get_device

current_directory = os.getcwd()
save_path = os.path.join(os.path.dirname(current_directory), "pretrained_models")
os.makedirs(save_path, exist_ok=True)
model_path = os.path.join(save_path, 'model_weights.pt')


In [21]:
import requests

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

In [22]:
file_id = '1IdaBtMSvtyzF0ByVaBHtvM0JYSXRExRX'
download_file_from_google_drive(file_id, 'model_weights.pt')

In [17]:
#@title Model

class MLP(nn.Module):

    def forward(self, x: T) -> T:
        return self.model(x)

    def __init__(self, sizes: Tuple[int, ...], bias=True, act=nn.Tanh):
        super(MLP, self).__init__()
        layers = []
        for i in range(len(sizes) -1):
            layers.append(nn.Linear(sizes[i], sizes[i + 1], bias=bias))
            if i < len(sizes) - 2:
                layers.append(act())
        self.model = nn.Sequential(*layers)


class ClipCaptionModel(nn.Module):

    #@functools.lru_cache #FIXME
    def get_dummy_token(self, batch_size: int, device: D) -> T:
        return torch.zeros(batch_size, self.prefix_length, dtype=torch.int64, device=device)

    def forward(self, tokens: T, prefix: T, mask: Optional[T] = None, labels: Optional[T] = None):
        embedding_text = self.gpt.transformer.wte(tokens)
        prefix_projections = self.clip_project(prefix).view(-1, self.prefix_length, self.gpt_embedding_size)
        #print(embedding_text.size()) #torch.Size([5, 67, 768])
        #print(prefix_projections.size()) #torch.Size([5, 1, 768])
        embedding_cat = torch.cat((prefix_projections, embedding_text), dim=1)
        if labels is not None:
            dummy_token = self.get_dummy_token(tokens.shape[0], tokens.device)
            labels = torch.cat((dummy_token, tokens), dim=1)
        out = self.gpt(inputs_embeds=embedding_cat, labels=labels, attention_mask=mask)
        return out

    def __init__(self, prefix_length: int, prefix_size: int = 512):
        super(ClipCaptionModel, self).__init__()
        self.prefix_length = prefix_length
        self.gpt = GPT2LMHeadModel.from_pretrained('gpt2')
        self.gpt_embedding_size = self.gpt.transformer.wte.weight.shape[1]
        if prefix_length > 10:  # not enough memory
            self.clip_project = nn.Linear(prefix_size, self.gpt_embedding_size * prefix_length)
        else:
            self.clip_project = MLP((prefix_size, (self.gpt_embedding_size * prefix_length) // 2, self.gpt_embedding_size * prefix_length))


class ClipCaptionPrefix(ClipCaptionModel):

    def parameters(self, recurse: bool = True):
        return self.clip_project.parameters()

    def train(self, mode: bool = True):
        super(ClipCaptionPrefix, self).train(mode)
        self.gpt.eval()
        return self

In [18]:
#@title Caption prediction

def generate_beam(model, tokenizer, beam_size: int = 5, prompt=None, embed=None,
                  entry_length=67, temperature=1., stop_token: str = '.'):

    model.eval()
    stop_token_index = tokenizer.encode(stop_token)[0]
    tokens = None
    scores = None
    device = next(model.parameters()).device
    seq_lengths = torch.ones(beam_size, device=device)
    is_stopped = torch.zeros(beam_size, device=device, dtype=torch.bool)
    with torch.no_grad():
        if embed is not None:
            generated = embed
        else:
            if tokens is None:
                tokens = torch.tensor(tokenizer.encode(prompt))
                tokens = tokens.unsqueeze(0).to(device)
                generated = model.gpt.transformer.wte(tokens)
        for i in range(entry_length):
            outputs = model.gpt(inputs_embeds=generated)
            logits = outputs.logits
            logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.0)
            logits = logits.softmax(-1).log()
            if scores is None:
                scores, next_tokens = logits.topk(beam_size, -1)
                generated = generated.expand(beam_size, *generated.shape[1:])
                next_tokens, scores = next_tokens.permute(1, 0), scores.squeeze(0)
                if tokens is None:
                    tokens = next_tokens
                else:
                    tokens = tokens.expand(beam_size, *tokens.shape[1:])
                    tokens = torch.cat((tokens, next_tokens), dim=1)
            else:
                logits[is_stopped] = -float(np.inf)
                logits[is_stopped, 0] = 0
                scores_sum = scores[:, None] + logits
                seq_lengths[~is_stopped] += 1
                scores_sum_average = scores_sum / seq_lengths[:, None]
                scores_sum_average, next_tokens = scores_sum_average.view(-1).topk(beam_size, -1)
                next_tokens_source = next_tokens // scores_sum.shape[1]
                seq_lengths = seq_lengths[next_tokens_source]
                next_tokens = next_tokens % scores_sum.shape[1]
                next_tokens = next_tokens.unsqueeze(1)
                tokens = tokens[next_tokens_source]
                tokens = torch.cat((tokens, next_tokens), dim=1)
                generated = generated[next_tokens_source]
                scores = scores_sum_average * seq_lengths
                is_stopped = is_stopped[next_tokens_source]
            next_token_embed = model.gpt.transformer.wte(next_tokens.squeeze()).view(generated.shape[0], 1, -1)
            generated = torch.cat((generated, next_token_embed), dim=1)
            is_stopped = is_stopped + next_tokens.eq(stop_token_index).squeeze()
            if is_stopped.all():
                break
    scores = scores / seq_lengths
    output_list = tokens.cpu().numpy()
    output_texts = [tokenizer.decode(output[:int(length)]) for output, length in zip(output_list, seq_lengths)]
    order = scores.argsort(descending=True)
    output_texts = [output_texts[i] for i in order]
    return output_texts


def generate2(
        model,
        tokenizer,
        tokens=None,
        prompt=None,
        embed=None,
        entry_count=1,
        entry_length=67,  # maximum number of words
        top_p=0.8,
        temperature=1.,
        stop_token: str = '.',
):
    model.eval()
    generated_num = 0
    generated_list = []
    stop_token_index = tokenizer.encode(stop_token)[0]
    filter_value = -float("Inf")
    device = next(model.parameters()).device

    with torch.no_grad():

        for entry_idx in trange(entry_count):
            if embed is not None:
                generated = embed
            else:
                if tokens is None:
                    tokens = torch.tensor(tokenizer.encode(prompt))
                    tokens = tokens.unsqueeze(0).to(device)

                generated = model.gpt.transformer.wte(tokens)

            for i in range(entry_length):

                outputs = model.gpt(inputs_embeds=generated)
                logits = outputs.logits
                logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.0)
                sorted_logits, sorted_indices = torch.sort(logits, descending=True)
                cumulative_probs = torch.cumsum(nnf.softmax(sorted_logits, dim=-1), dim=-1)
                sorted_indices_to_remove = cumulative_probs > top_p
                sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[
                                                    ..., :-1
                                                    ].clone()
                sorted_indices_to_remove[..., 0] = 0

                indices_to_remove = sorted_indices[sorted_indices_to_remove]
                logits[:, indices_to_remove] = filter_value
                next_token = torch.argmax(logits, -1).unsqueeze(0)
                next_token_embed = model.gpt.transformer.wte(next_token)
                if tokens is None:
                    tokens = next_token
                else:
                    tokens = torch.cat((tokens, next_token), dim=1)
                generated = torch.cat((generated, next_token_embed), dim=1)
                if stop_token_index == next_token.item():
                    break

            output_list = list(tokens.squeeze().cpu().numpy())
            output_text = tokenizer.decode(output_list)
            generated_list.append(output_text)

    return generated_list[0]

In [7]:
#@title GPU/CPU


is_gpu = True #@param {type:"boolean"}  


In [8]:
#@title CLIP model + GPT2 tokenizer

device = CUDA(0) if is_gpu else "cpu"
clip_model, preprocess = clip.load("ViT-B/32", device=device, jit=False)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

In [25]:
#@title Load model weights


prefix_length = 10

model = ClipCaptionModel(prefix_length)

model.load_state_dict(torch.load(model_path, map_location=CPU)) 

model = model.eval() 
device = CUDA(0) if is_gpu else "cpu"
model = model.to(device)


In [26]:
def generate_caption(img_file, display_img=False, use_beam_search=False):
    image = io.imread(img_file)
    pil_image = PIL.Image.fromarray(image)
    # pil_img = Image(filename=UPLOADED_FILE)
    if display_img:
        display(pil_image.resize((512, 512)))
    image = preprocess(pil_image).unsqueeze(0).to(device)
    with torch.no_grad():
        # if type(model) is ClipCaptionE2E:
        #     prefix_embed = model.forward_image(image)
        # else:
        prefix = clip_model.encode_image(image).to(device, dtype=torch.float32)
        prefix_embed = model.clip_project(prefix).reshape(1, prefix_length, -1)
    if use_beam_search:
        generated_text_prefix = generate_beam(model, tokenizer, embed=prefix_embed)[0]
    else:
        generated_text_prefix = generate2(model, tokenizer, embed=prefix_embed)


    return generated_text_prefix, prefix

In [27]:
def calc_style_embedding(img_file, display_img=False, use_beam_search=False, device='cpu', manual_caption=None):
    caption, clip_img_rep = generate_caption(img_file, display_img=display_img, use_beam_search=use_beam_search)
    if manual_caption is not None:
        caption = manual_caption
    with torch.no_grad():
        caption_tokenized = clip.tokenize(caption, truncate=True).to(device)
        clip_caption_rep = clip_model.encode_text(caption_tokenized).to(device, dtype=torch.float32)
    style = clip_img_rep - clip_caption_rep
    return style, caption

In [29]:
from sklearn.neighbors import NearestNeighbors

def rank_by_style_similarity(input_file, target_files, device, manual_captions=None, use_beam_search=False):
    image = io.imread(input_file)
    pil_image = PIL.Image.fromarray(image)
    display(pil_image.resize((512, 512)))
    nbrs = NearestNeighbors(metric='cosine', n_neighbors=20)
    if manual_captions is not None:
        manual_caption = manual_captions.loc[manual_captions['file_path']==input_file, 'caption'].values[0]
        input_style, input_caption = calc_style_embedding(input_file, use_beam_search=use_beam_search, device=device, manual_caption=manual_caption)
    else:
        input_style, input_caption = calc_style_embedding(input_file, use_beam_search=use_beam_search, device=device)
    input_style = input_style.cpu().numpy()
    print(input_caption)
    target_styles = []
    target_captions = []
    for target_file in target_files:
        if manual_captions is not None:
            manual_caption = manual_captions.loc[manual_captions['file_path']==target_file, 'caption'].values[0]
            target_style, target_caption = calc_style_embedding(input_file, device=device, manual_caption=manual_caption)
        else:
            target_style, target_caption = calc_style_embedding(target_file, device=device)
        target_style = target_style.squeeze(dim=0).cpu().numpy()
        target_styles.append(target_style)
        target_captions.append(target_caption)
        
    nbrs.fit(np.stack(np.array(target_styles)))
    nearest_scores, nearest_imgs = nbrs.kneighbors(input_style)
    for idx in nearest_imgs.squeeze():
        image = io.imread(target_files[idx])
        pil_image = PIL.Image.fromarray(image)
        print(target_captions[idx])
        display(pil_image.resize((512, 512)))

In [30]:
def rank_by_clip_similarity(input_file, target_files, device):
    nbrs = NearestNeighbors(metric='cosine', n_neighbors=20)
    
    image = io.imread(input_file)
    pil_image = PIL.Image.fromarray(image)
    display(pil_image.resize((512, 512)))
    image = preprocess(pil_image).unsqueeze(0).to(device)
    input_clip_embedding = clip_model.encode_image(image).to(device, dtype=torch.float32)
    input_clip_embedding = input_clip_embedding.detach().numpy()
    print(input_clip_embedding.shape)
    
    target_clip_embeddings = []
    for target_file in target_files:
        image = io.imread(target_file)
        pil_image = PIL.Image.fromarray(image)
        image = preprocess(pil_image).unsqueeze(0).to(device)
        target_clip_embedding = clip_model.encode_image(image).to(device, dtype=torch.float32)
        target_clip_embedding = target_clip_embedding.detach().numpy().squeeze(0)
        target_clip_embeddings.append(target_clip_embedding)
    print(np.stack(np.array(target_clip_embeddings)).shape)
    nbrs.fit(np.stack(np.array(target_clip_embeddings)))
    nearest_scores, nearest_imgs = nbrs.kneighbors(input_clip_embedding)
    for idx in nearest_imgs.squeeze():
        image = io.imread(target_files[idx])
        pil_image = PIL.Image.fromarray(image)
        display(pil_image.resize((512, 512)))

In [155]:
input_account = 'mrwhisper'
input_file = f'{os.path.dirname(current_directory)}/images/{input_account}_imgs/j_kx3eq39l15jy4our1a.d1429bcdebb1d0268d66ae8c2eee24ed4c2e708b.file_kx3pbj60btr4fcknj.post_image_url.jpg'

In [156]:
target_accounts = ['raylivez']
target_files = []
for target_account in target_accounts:
    target_files += glob.glob(f'{os.path.dirname(current_directory)}/images/{target_account}_imgs/*.jpg')

In [236]:
# rank_by_style_similarity(input_file, target_files, device, use_beam_search=True)

In [237]:
# rank_by_clip_similarity(input_file, target_files, device)

In [150]:
input_account = 'mrwhisper'
input_file = f'{os.path.dirname(current_directory)}/images/{input_account}_imgs/j_kx3eq39l15jy4our1a.d1429bcdebb1d0268d66ae8c2eee24ed4c2e708b.file_kx3pbiwmsdxc15tym.post_image_url.jpg'

In [147]:
target_accounts = ['raylivez']
target_files = []
for target_account in target_accounts:
    target_files += glob.glob(f'{os.path.dirname(current_directory)}/images/{target_account}_imgs/*.jpg')

In [235]:
# rank_by_style_similarity(input_file, target_files, device, use_beam_search=True)

In [125]:
input_account = 'raylivez'
input_file = f'{os.path.dirname(current_directory)}/images/{input_account}_imgs/j_kx3eq39l15jy4our1a.afa2c1adc22efdff2add470abea08b0ced30748d.file_kx3ija16kou2y0nz9.post_image_url.jpg'

In [126]:
target_accounts = ['raylivez']
target_files = []
for target_account in target_accounts:
    target_files += glob.glob(f'{os.path.dirname(current_directory)}/images/{target_account}_imgs/*.jpg')

In [234]:
# rank_by_style_similarity(input_file, target_files, device, use_beam_search=True)

In [130]:
input_account = 'dani_fitlifestyle'
input_file = f'{os.path.dirname(current_directory)}/images/{input_account}_imgs/j_kx3eq39l15jy4our1a.65e9cd7e0a2138f8e2cdf1a33d6b5e0688796470.file_kx3jwivi7y3c5gluu.post_image_url.jpg'

In [131]:
target_accounts = ['dani_fitlifestyle','liviegrace']
target_files = []
for target_account in target_accounts:
    if target_account == input_account:
        target_files += glob.glob(f'{os.path.dirname(current_directory)}/images/{target_account}_imgs/*.jpg')[:2]
    else:
        target_files += glob.glob(f'{os.path.dirname(current_directory)}/images/{target_account}_imgs/*.jpg')

In [233]:
# rank_by_style_similarity(input_file, target_files, device, use_beam_search=True)

In [232]:
# rank_by_clip_similarity(input_file, target_files, device)

In [115]:
input_account = 'sweetartscandy'
input_file = f'{os.path.dirname(current_directory)}/images/{input_account}_imgs/j_kx3eq39l15jy4our1a.7318572adfee524b808153d2572a21359298f705.file_kx3gu5ec21err4syd2.post_image_url.jpg'

In [116]:
target_accounts = ['liviegrace','frootloops']
target_files = []
for target_account in target_accounts:
    if target_account == 'frootloops':
        target_files += glob.glob(f'{os.path.dirname(current_directory)}/images/{target_account}_imgs/*.jpg')[:4]
    else:
        target_files += glob.glob(f'{os.path.dirname(current_directory)}/images/{target_account}_imgs/*.jpg')

In [231]:
# rank_by_style_similarity(input_file, target_files, device, use_beam_search=True)

In [143]:
# account_name = 'liviegrace'
# for file_ in glob.glob(f'{os.path.dirname(current_directory)}/images/{account_name}_imgs/*.jpg'):
#     style = calc_style_embedding(file_, display_img=False, use_beam_search=True, device=device)
#     print(style.shape)

In [167]:
# import glob
# account_name = 'liviegrace'
# for file_ in glob.glob(f'{os.path.dirname(current_directory)}/images/{account_name}_imgs/*.jpg'):
#     text, _ = generate_caption(img_file=file_, display_img=True, use_beam_search=True)
#     print(text)
#     print(file_)

In [81]:
import pandas as pd
df = pd.read_csv('liviegrace_manual_captions.csv',names=['file_path','caption'])

In [89]:
input_idx = 3
input_file = df.loc[3,'file_path']
target_files = df.loc[df.index != 3,'file_path'].tolist()

In [229]:
# rank_by_style_similarity(input_file, target_files, device, df)

# COCO Dataset

In [181]:
coco_test = os.listdir(f'{os.path.dirname(current_directory)}/images/test2017')

In [182]:
coco_test = [f'{os.path.dirname(current_directory)}/images/test2017/{file}' for file in coco_test]

In [246]:
import random
coco_sample = random.sample(coco_test, 1000)

In [245]:
# from pydrive.drive import GoogleDrive
# from pydrive.auth import GoogleAuth
   
# # For using listdir()
# import os
   
  
# # Below code does the authentication
# # part of the code
# gauth = GoogleAuth()
  
# # Creates local webserver and auto
# # handles authentication.
# gauth.LocalWebserverAuth()       
# drive = GoogleDrive(gauth)
   
# # replace the value of this variable
# # with the absolute path of the directory
# path = f'{os.path.dirname(current_directory)}/results'   
   
# # iterating thought all the files/folder
# # of the desired directory
# for x in os.listdir(path):
#     f = drive.CreateFile({'title': x, 'parents': [{'id': '1SVuS_UiCEs_cD7LBwoaVO224N7yXb2LT'}]})
#     f.SetContentFile(os.path.join(path, x))
#     f.Upload()
  
#     # Due to a known bug in pydrive if we 
#     # don't empty the variable used to
#     # upload the files to Google Drive the
#     # file stays open in memory and causes a
#     # memory leak, therefore preventing its 
#     # deletion
#     f = None

In [247]:
from sklearn.metrics.pairwise import cosine_similarity
def calc_style_similarities(files):
    img_styles = []
    captions = []
    for img_file in files:
        style, caption =  calc_style_embedding(img_file, display_img=False, use_beam_search=False, device='cpu', manual_caption=None)
        style = style.squeeze(dim=0).cpu().numpy()
        img_styles.append(style)
        captions.append(caption)
    sim_mtx = cosine_similarity(np.stack(np.array(img_styles)))
    return sim_mtx, captions

In [248]:
coco_style_sim_mtx, captions = calc_style_similarities(coco_sample)

100%|██████████████████████████████| 1/1 [00:01<00:00,  1.24s/it]


In [252]:
coco_style_sim_mtx.shape

(1000, 1000)

In [250]:
def calc_clip_similarities(files):
    clip_embeddings = []
    for img_file in files:
        image = io.imread(img_file)
        pil_image = PIL.Image.fromarray(image)
        image = preprocess(pil_image).unsqueeze(0).to(device)
        clip_embedding = clip_model.encode_image(image).to(device, dtype=torch.float32)
        clip_embedding = clip_embedding.detach().numpy().squeeze(0)
        clip_embeddings.append(clip_embedding)
    sim_mtx = cosine_similarity(np.stack(np.array(clip_embeddings)))
    return sim_mtx

In [251]:
coco_clip_sim_mtx = calc_clip_similarities(coco_sample)

In [253]:
coco_style_sim_mtx.shape

(1000, 1000)

In [254]:
def compare_clip_style_sims(img_files_lst, style_sim_mtx, clip_sym_mtx, out_fold):
    from PIL import Image
    for i in range(len(img_files_lst)):
        style_sim_mtx[i,i] = -1
        clip_sym_mtx[i,i] = -1
        style_idx_most_sim = style_sim_mtx[i].argmax()
        clip_idx_most_sim = clip_sym_mtx[i].argmax()
        if style_idx_most_sim == clip_idx_most_sim:
            continue
        images = [Image.open(img_files_lst[i])]
        images.append(Image.open(img_files_lst[style_idx_most_sim]))
        images.append(Image.open(img_files_lst[clip_idx_most_sim]))
        widths, heights = zip(*(i.size for i in images))

        total_width = sum(widths)
        max_height = max(heights)

        new_im = Image.new('RGB', (total_width, max_height))

        x_offset = 0
        for im in images:
            new_im.paste(im, (x_offset,0))
            x_offset += im.size[0]

        new_im.save(f'{os.path.dirname(current_directory)}/results/1000_imgs/{img_files_lst[i].split("/")[-1]}')
        
compare_clip_style_sims(coco_sample, coco_style_sim_mtx, coco_clip_sim_mtx)

# Unsplash Data

In [31]:
import requests
from nltk.sentiment import SentimentIntensityAnalyzer

In [215]:
path = f'{current_directory.replace("notebooks","")}data/unsplash-research-dataset-lite-latest'

In [35]:
documents = ['photos', 'keywords', 'collections', 'conversions', 'colors']
datasets = {}

for doc in documents:
    files = glob.glob(path + doc + ".tsv*")
    print(files)
    subsets = []
    for filename in files:
        df = pd.read_csv(filename, sep='\t', header=0)
        subsets.append(df)

    datasets[doc] = pd.concat(subsets, axis=0, ignore_index=True)

['/Users/kgaiger/PycharmProjects/clip_new/CLIP_prefix_caption-1/data/unsplash-research-dataset-lite-latest/photos.tsv000']
['/Users/kgaiger/PycharmProjects/clip_new/CLIP_prefix_caption-1/data/unsplash-research-dataset-lite-latest/keywords.tsv000']
['/Users/kgaiger/PycharmProjects/clip_new/CLIP_prefix_caption-1/data/unsplash-research-dataset-lite-latest/collections.tsv000']
['/Users/kgaiger/PycharmProjects/clip_new/CLIP_prefix_caption-1/data/unsplash-research-dataset-lite-latest/conversions.tsv000']
['/Users/kgaiger/PycharmProjects/clip_new/CLIP_prefix_caption-1/data/unsplash-research-dataset-lite-latest/colors.tsv000']


In [96]:
indices = datasets['photos'].head(10000).index

In [62]:
failures = 0

In [99]:
for index in tqdm(indices):
    try:
        response = requests.get(datasets['photos'].loc[index,'photo_image_url'])
        file = open(f'{path}/images/{index}.png', "wb")
        file.write(response.content)
        file.close()
    except:
        print('fail!')
        failures += 1

  2%|▍                  | 157/7000 [28:45<51:53:31, 27.30s/it]

fail!


  2%|▍                  | 158/7000 [29:15<53:25:57, 28.11s/it]

fail!


  2%|▍                  | 159/7000 [29:45<54:30:32, 28.68s/it]

...

 70%|███████████▏    | 4912/7000 [2:53:01<17:12:04, 29.66s/it]

fail!


 70%|███████████▏    | 4913/7000 [2:53:31<17:15:19, 29.77s/it]

fail!


100%|███████████████████| 7000/7000 [3:54:54<00:00,  2.01s/it]


In [102]:
tested = datasets['photos'].loc[indices]

In [103]:
tested['style_emb'] = np.nan 
tested['caption'] = np.nan
tested['neg'] = 0
tested['neu'] = 0
tested['pos'] = 0
tested['compound'] = 0       

In [104]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/kgaiger/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [105]:
sia = SentimentIntensityAnalyzer()

In [108]:
int(os.listdir(f'{path}/images')[0].replace('.png',''))

3975

In [113]:
styles_dict = {}
for img_file in os.listdir(f'{path}/images'):
    try:
        style, caption = calc_style_embedding(f'{path}/images/{img_file}', display_img=False, use_beam_search=False, device='cpu', manual_caption=None)
        style = np.squeeze(style.cpu().numpy(),axis=0)
        # tested['style_emb'] = tested['style_emb'].astype(object)
        index = int(img_file.replace('.png',''))
        # tested.loc[index, 'style_emb'] = style
        styles_dict[index] = style
        tested.loc[index, 'caption'] = caption
        polairty_scores = sia.polarity_scores(caption)
        for k in polairty_scores.keys():
            tested.loc[index, k] = polairty_scores[k]        
        if index % 100 == 0:
            print(index)
    except:
        print('fail!')
        continue

100%|███████████████████████████| 1/1 [00:00<00:00,  1.00it/s]


9900


100%|███████████████████████████| 1/1 [00:01<00:00,  1.06s/it]


6400


100%|███████████████████████████| 1/1 [00:01<00:00,  1.02s/it]


...

100%|███████████████████████████| 1/1 [00:01<00:00,  1.13s/it]


9500


100%|███████████████████████████| 1/1 [00:01<00:00,  1.12s/it]


6800


100%|███████████████████████████| 1/1 [00:01<00:00,  1.12s/it]


In [ ]:
tested = tested[tested['style'].notnull()]

In [152]:
tested = tested.reset_index()

In [ ]:
tested['style'] = tested['index'].map(styles_dict)

In [209]:
def calc_clip_embedding(img_file):
    try:
        image = io.imread(img_file)
        pil_image = PIL.Image.fromarray(image)
        image = preprocess(pil_image).unsqueeze(0).to(device)
        with torch.no_grad():
            clip_emb = clip_model.encode_image(image).to(device, dtype=torch.float32)
            clip_emb = clip_emb.squeeze(dim=0).cpu().numpy()
    except:
        clip_emb = None
    return clip_emb

tested['clip']= tested.progress_apply(lambda row: calc_clip_embedding(f'{path}/images/{str(row.name)}.png'), axis=1)

 27%|█████▊                | 2655/9976 [29:21<1:00:54,  2.00it/s]/opt/anaconda3/envs/clip_prefix_caption/lib/python3.9/site-packages/PIL/Image.py:2918: DecompressionBombWarning: Image size (96012000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
 40%|████████▉             | 4030/9976 [44:27<1:01:03,  1.62it/s]/opt/anaconda3/envs/clip_prefix_caption/lib/python3.9/site-packages/PIL/TiffImagePlugin.py:625: UserWarning: Metadata Warning, tag 33723 had too many entries: 28, expected 1
  warnings.warn(
 41%|████████▉             | 4045/9976 [44:38<1:13:56,  1.34it/s]/opt/anaconda3/envs/clip_prefix_caption/lib/python3.9/site-packages/PIL/TiffImagePlugin.py:819: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))
 50%|███████████           | 5005/9976 [55:04<1:08:45,  1.21it/s]/opt/anaconda3/envs/clip_prefix_caption/lib/python3.9/site-packages/PIL/TiffImagePlugin.py:625: UserWarning: Metadata 

In [212]:
tested.to_csv('tested.csv',index=False)

In [141]:
style_sim_mtx = cosine_similarity(np.stack(np.array(tested['style'].tolist())))

In [154]:
def find_most_sim_emb(input_img_idx, sim_mtx):
    sim_mtx[input_img_idx,input_img_idx] = -1
    idx_most_sim = sim_mtx[input_img_idx].argmax()
    return idx_most_sim

In [155]:
tested['most_sim_style_idx'] = tested.apply(lambda row: find_most_sim_emb(row.name, style_sim_mtx), axis=1)

In [193]:
tested['most_sim_style_url'] = tested.apply(lambda row: tested.loc[row['most_sim_style_idx'],'photo_url'], axis=1)

In [202]:
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', -1)  # or 199

/var/folders/lm/r78yjp1d4094sfws8gn69l6w0000gq/T/ipykernel_31869/1999909185.py:3: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)  # or 199


In [207]:
tested.loc[120]

index                             120                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [226]:
from PIL import Image
import os, sys

path = '/Users/kgaiger/PycharmProjects/clip_new/CLIP_prefix_caption-1/data/unsplash-research-dataset-lite-latest/images/'
new_path = '/Users/kgaiger/PycharmProjects/clip_new/CLIP_prefix_caption-1/data/unsplash-research-dataset-lite-latest/images_1000*1000/'

def resize():
    for item in tqdm(os.listdir(path)):
        if os.path.isfile(path+item):
            try:
                im = Image.open(path+item)
                f, e = os.path.splitext(path+item)
                imResize = im.resize((512,512), Image.ANTIALIAS)
                imResize.save(new_path + item, 'PNG', quality=90)
            except:
                print('fail!')
                continue
resize()   

 31%|██████▉               | 3141/9990 [27:27<1:06:36,  1.71it/s]

fail!


 44%|██████████▌             | 4394/9990 [38:05<53:50,  1.73it/s]

fail!


 50%|███████████▉            | 4994/9990 [43:14<23:48,  3.50it/s]/opt/anaconda3/envs/clip_prefix_caption/lib/python3.9/site-packages/PIL/Image.py:2918: DecompressionBombWarning: Image size (99991727 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
 56%|█████████████▎          | 5555/9990 [48:00<35:38,  2.07it/s]/opt/anaconda3/envs/clip_prefix_caption/lib/python3.9/site-packages/PIL/Image.py:2918: DecompressionBombWarning: Image size (94212096 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
 81%|█████████████████▉    | 8123/9990 [1:10:03<10:16,  3.03it/s]/opt/anaconda3/envs/clip_prefix_caption/lib/python3.9/site-packages/PIL/Image.py:2918: DecompressionBombWarning: Image size (96768910 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
 88%|███████████████████▎  | 8762/9990 [1:15:24<12:32,  1.63it/s]

fail!


 94%|████████████████████▋ | 9387/9990 [1:20:46<04:38,  2.17it/s]

fail!


100%|██████████████████████| 9990/9990 [1:25:59<00:00,  1.94it/s]


In [224]:
os.listdir(path)[0]

'3975.png'